In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from math import radians, cos, sin, asin, sqrt

In [3]:
df = pd.read_csv('TimeLoc_Dataframe.csv')
vehicle_2_df = pd.read_csv('vehicle_2_TimeLoc_Dataframe.csv')
vehicle_2_df

,idle_start_time,idle_end_time,idle_duration_hr,idle_start_time_mdt,idle_end_time_mdt,idle_start_hour_mdt,timestamp,idle_start_hour,week_start_date,week_number,week_end_date,lat,lon
0,2025-05-08 14:39:11.010000+00:00,2025-05-08 23:01:11+00:00,8.366664,2025-05-08 08:39:11.010000-06:00,2025-05-08 17:01:11-06:00,8.016667,2025-05-08 08:39:11.010,8,2025-05-06,1,2025-05-12,51.084428,-114.131383
1,2025-05-08 23:27:35+00:00,2025-05-09 19:13:29+00:00,19.765000,2025-05-08 17:27:35-06:00,2025-05-09 13:13:29-06:00,17.216667,2025-05-08 17:27:35.000,17,2025-05-06,1,2025-05-12,51.037898,-114.174673
2,2025-05-09 19:20:16.010000+00:00,2025-05-09 20:12:26+00:00,0.869442,2025-05-09 13:20:16.010000-06:00,2025-05-09 14:12:26-06:00,13.200000,2025-05-09 13:20:16.010,13,2025-05-06,1,2025-05-12,51.020670,-114.167112
3,2025-05-09 20:16:17.010000+00:00,2025-05-09 20:22:19.010000+00:00,0.100556,2025-05-09 14:16:17.010000-06:00,2025-05-09 14:22:19.010000-06:00,14.366667,2025-05-09 14:16:17.010,14,2025-05-06,1,2025-05-12,51.018893,-114.173660
4,2025-05-09 20:43:31+00:00,2025-05-09 20:46:51+00:00,0.055556,2025-05-09 14:43:31-06:00,2025-05-09 14:46:51-06:00,14.766667,2025-05-09 14:43:31.000,14,2025-05-06,1,2025-05-12,50.940008,-114.144882
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,2025-06-12 22:32:09.010000+00:00,2025-06-13 19:59:03.010000+00:00,21.448333,2025-06-12 16:32:09.010000-06:00,2025-06-13 13:59:03.010000-06:00,16.983333,2025-06-12 16:32:09.010,16,2025-06-10,6,2025-06-16,51.037908,-114.174697
72,2025-06-13 20:20:15+00:00,2025-06-13 20:37:01+00:00,0.279444,2025-06-13 14:20:15-06:00,2025-06-13 14:37:01-06:00,14.616667,2025-06-13 14:20:15.000,14,2025-06-10,6,2025-06-16,51.085468,-114.123373
73,2025-06-13 20:54:24+00:00,2025-06-13 21:38:09+00:00,0.729167,2025-06-13 14:54:24-06:00,2025-06-13 15:38:09-06:00,14.633333,2025-06-13 14:54:24.000,14,2025-06-10,6,2025-06-16,51.019005,-114.161488
74,2025-06-13 21:43:02+00:00,2025-06-13 23:13:17+00:00,1.504167,2025-06-13 15:43:02-06:00,2025-06-13 17:13:17-06:00,15.216667,2025-06-13 15:43:02.000,15,2025-06-10,6,2025-06-16,51.021412,-114.166585


In [4]:
df = df.sort_values('idle_start_time_mdt').reset_index(drop=True)
vehicle_2_df = vehicle_2_df.sort_values('idle_start_time_mdt').reset_index(drop=True)
vehicle_2_df

,idle_start_time,idle_end_time,idle_duration_hr,idle_start_time_mdt,idle_end_time_mdt,idle_start_hour_mdt,timestamp,idle_start_hour,week_start_date,week_number,week_end_date,lat,lon
0,2025-05-08 14:39:11.010000+00:00,2025-05-08 23:01:11+00:00,8.366664,2025-05-08 08:39:11.010000-06:00,2025-05-08 17:01:11-06:00,8.016667,2025-05-08 08:39:11.010,8,2025-05-06,1,2025-05-12,51.084428,-114.131383
1,2025-05-08 23:27:35+00:00,2025-05-09 19:13:29+00:00,19.765000,2025-05-08 17:27:35-06:00,2025-05-09 13:13:29-06:00,17.216667,2025-05-08 17:27:35.000,17,2025-05-06,1,2025-05-12,51.037898,-114.174673
2,2025-05-09 19:20:16.010000+00:00,2025-05-09 20:12:26+00:00,0.869442,2025-05-09 13:20:16.010000-06:00,2025-05-09 14:12:26-06:00,13.200000,2025-05-09 13:20:16.010,13,2025-05-06,1,2025-05-12,51.020670,-114.167112
3,2025-05-09 20:16:17.010000+00:00,2025-05-09 20:22:19.010000+00:00,0.100556,2025-05-09 14:16:17.010000-06:00,2025-05-09 14:22:19.010000-06:00,14.366667,2025-05-09 14:16:17.010,14,2025-05-06,1,2025-05-12,51.018893,-114.173660
4,2025-05-09 20:43:31+00:00,2025-05-09 20:46:51+00:00,0.055556,2025-05-09 14:43:31-06:00,2025-05-09 14:46:51-06:00,14.766667,2025-05-09 14:43:31.000,14,2025-05-06,1,2025-05-12,50.940008,-114.144882
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,2025-06-12 22:32:09.010000+00:00,2025-06-13 19:59:03.010000+00:00,21.448333,2025-06-12 16:32:09.010000-06:00,2025-06-13 13:59:03.010000-06:00,16.983333,2025-06-12 16:32:09.010,16,2025-06-10,6,2025-06-16,51.037908,-114.174697
72,2025-06-13 20:20:15+00:00,2025-06-13 20:37:01+00:00,0.279444,2025-06-13 14:20:15-06:00,2025-06-13 14:37:01-06:00,14.616667,2025-06-13 14:20:15.000,14,2025-06-10,6,2025-06-16,51.085468,-114.123373
73,2025-06-13 20:54:24+00:00,2025-06-13 21:38:09+00:00,0.729167,2025-06-13 14:54:24-06:00,2025-06-13 15:38:09-06:00,14.633333,2025-06-13 14:54:24.000,14,2025-06-10,6,2025-06-16,51.019005,-114.161488
74,2025-06-13 21:43:02+00:00,2025-06-13 23:13:17+00:00,1.504167,2025-06-13 15:43:02-06:00,2025-06-13 17:13:17-06:00,15.216667,2025-06-13 15:43:02.000,15,2025-06-10,6,2025-06-16,51.021412,-114.166585


### Feature Engineering

In [6]:
df = df.copy()

# Convert 
df['idle_start_time_mdt'] = pd.to_datetime(df['idle_start_time_mdt'], format='mixed')
df['idle_end_time_mdt'] = pd.to_datetime(df['idle_end_time_mdt'], format='mixed')
vehicle_2_df['idle_start_time_mdt'] = pd.to_datetime(vehicle_2_df['idle_start_time_mdt'], format='mixed')
vehicle_2_df['idle_end_time_mdt'] = pd.to_datetime(vehicle_2_df['idle_end_time_mdt'], format='mixed')

# Encode cyclical start time features
df['start_hour'] = df['idle_start_time_mdt'].dt.hour + df['idle_start_time_mdt'].dt.minute / 60
df['start_hour_sin'] = np.sin(2 * np.pi * df['start_hour'] / 24)
df['start_hour_cos'] = np.cos(2 * np.pi * df['start_hour'] / 24)
vehicle_2_df['start_hour'] = vehicle_2_df['idle_start_time_mdt'].dt.hour + vehicle_2_df['idle_start_time_mdt'].dt.minute / 60
vehicle_2_df['start_hour_sin'] = np.sin(2 * np.pi * vehicle_2_df['start_hour'] / 24)
vehicle_2_df['start_hour_cos'] = np.cos(2 * np.pi * vehicle_2_df['start_hour'] / 24)

# Encode cyclical end time features
df['end_hour'] = df['idle_end_time_mdt'].dt.hour + df['idle_end_time_mdt'].dt.minute / 60
df['end_hour_sin'] = np.sin(2 * np.pi * df['end_hour'] / 24)
df['end_hour_cos'] = np.cos(2 * np.pi * df['end_hour'] / 24)
vehicle_2_df['end_hour'] = vehicle_2_df['idle_end_time_mdt'].dt.hour + vehicle_2_df['idle_end_time_mdt'].dt.minute / 60
vehicle_2_df['end_hour_sin'] = np.sin(2 * np.pi * vehicle_2_df['end_hour'] / 24)
vehicle_2_df['end_hour_cos'] = np.cos(2 * np.pi * vehicle_2_df['end_hour'] / 24)

# Calculate duration
df['idle_duration_minutes'] = (df['idle_end_time_mdt'] - df['idle_start_time_mdt']).dt.total_seconds() / 60.0
vehicle_2_df['idle_duration_minutes'] = (vehicle_2_df['idle_end_time_mdt'] - vehicle_2_df['idle_start_time_mdt']).dt.total_seconds() / 60.0

# Final feature set
feature_cols = ['start_hour_sin', 'start_hour_cos', 'end_hour_sin', 'end_hour_cos', 'idle_duration_minutes']
target_cols = ['lat', 'lon']

### Normalize features

In [8]:
# Separate scalers for input features and target
feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()

# Fit and transform
df[feature_cols] = feature_scaler.fit_transform(df[feature_cols])
df[target_cols] = target_scaler.fit_transform(df[target_cols])
vehicle_2_df[feature_cols] = feature_scaler.fit_transform(vehicle_2_df[feature_cols])
vehicle_2_df[target_cols] = target_scaler.fit_transform(vehicle_2_df[target_cols])

### Create sequences for LSTM

In [11]:
sequence_length = 30

X, y = [], []
data = df[feature_cols + target_cols].to_numpy()

for i in range(len(data) - sequence_length):
    X.append(data[i:i+sequence_length, :len(feature_cols)])   # Features
    y.append(data[i+sequence_length, :2])                     # Next location (lat, lon)

X = np.array(X)
y = np.array(y)


sequence_length = 30

X_vehicle_2, y_vehicle_2 = [], []
vehicle_2_data = df[feature_cols + target_cols].to_numpy()

for i in range(len(vehicle_2_data) - sequence_length):
    X_vehicle_2.append(vehicle_2_data[i:i+sequence_length, :len(feature_cols)])  # Input features
    y_vehicle_2.append(vehicle_2_data[i+sequence_length, :2])  # lat, lon as target

X_vehicle_2 = np.array(X_vehicle_2)
y_vehicle_2 = np.array(y_vehicle_2)

### Train-test split

In [14]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Build and train LSTM model

In [16]:
model = Sequential([
    LSTM(128, activation='relu', input_shape=(X.shape[1], X.shape[2])),
    Dense(32, activation='relu'),
    Dense(2)  # Predict [lat, lon]
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

# Train
history = model.fit(X, y, epochs=20, batch_size=32, validation_split=0.2)

C:\Users\omoni\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 128)                 │          68,608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 32)                  │           4,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │              66 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 72,802 (284.38 KB)

 Trainable params: 72,802 (284.38 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 81ms/step - loss: 0.1730 - mae: 0.3085 - val_loss: 0.0894 - val_mae: 0.2116
Epoch 2/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0913 - mae: 0.2263 - val_loss: 0.0758 - val_mae: 0.2474
Epoch 3/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0699 - mae: 0.2272 - val_loss: 0.0577 - val_mae: 0.2001
Epoch 4/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0628 - mae: 0.2022 - val_loss: 0.0531 - val_mae: 0.2001
Epoch 5/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0514 - mae: 0.1866 - val_loss: 0.0490 - val_mae: 0.1920
Epoch 6/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0486 - mae: 0.1782 - val_loss: 0.0380 - val_mae: 0.1638
Epoch 7/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0394 - mae: 0.1524 - val_loss: 0.0318 - val_mae: 0.1477
Epoch 8/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0315 - mae: 0.1336 - val_loss: 0.0211 - val_mae: 0.1130
Epoch 9/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0208 - mae: 0.1002 - 

### Evaluate Model on Separate Vehicle Data

In [18]:
# Basic metrics
loss, mae = model.evaluate(X_vehicle_2, y_vehicle_2)
print(f"Test MAE (normalized): {mae:.4f}")

# Predict
y_pred = model.predict(X_vehicle_2)

# Inverse transform predictions and actuals
y_test_inv = target_scaler.inverse_transform(y_vehicle_2)
y_pred_inv = target_scaler.inverse_transform(y_pred)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0049 - mae: 0.0500
Test MAE (normalized): 0.0497
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


### Haversine Distance Evaluation

In [20]:
from haversine import haversine, Unit

y_pred = model.predict(X_vehicle_2)
y_test_inv = target_scaler.inverse_transform(y_vehicle_2)
y_pred_inv = target_scaler.inverse_transform(y_pred)

def haversine_errors(y_true, y_pred):
    return np.mean([
        haversine(tuple(true), tuple(pred), unit=Unit.METERS)
        for true, pred in zip(y_true, y_pred)
    ])

mean_haversine = haversine_errors(y_test_inv, y_pred_inv)
print(f"Mean Haversine Error: {mean_haversine:.2f} meters")

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Mean Haversine Error: 1907.95 meters
